# Kaggle Competition - h2o

In [1]:
import pandas as pd
import numpy as np
import h2o

from process_fun import *
from train_fun import *

from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /usr/local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/ys/69ly40l535g54jmcdws91jbh0000gn/T/tmprjzgnr3o
  JVM stdout: /var/folders/ys/69ly40l535g54jmcdws91jbh0000gn/T/tmprjzgnr3o/h2o_ana_started_from_python.out
  JVM stderr: /var/folders/ys/69ly40l535g54jmcdws91jbh0000gn/T/tmprjzgnr3o/h2o_ana_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_ana_fgel9v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


### Preparing data

In [2]:
d_train = pd.read_csv("../input/diamonds-datamad0320/diamonds_train.csv")
d_test = pd.read_csv("../input/diamonds-datamad0320/diamonds_test.csv")

In [3]:
d_train.drop(columns=['x', 'y', 'z','Unnamed: 0'],inplace=True)

cutlis=['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
to_num_con(d_train, 'cut', cutlis)

colorlis = ['J','I','H','G','F','E','D']
to_num_con(d_train, 'color', colorlis)

clalis = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']
to_num_con(d_train, 'clarity', clalis) 

d_train.head(3)

,carat,cut,color,clarity,depth,table,price
0,1.21,5,3,4,63.0,57.0,6134
1,0.28,3,7,6,64.0,56.0,532
2,0.42,4,5,5,61.2,58.0,1103


In [4]:
d_test.drop(columns=['x', 'y', 'z','Unnamed: 0'],inplace=True)

to_num_con(d_test, 'cut', cutlis)
to_num_con(d_test, 'color', colorlis)
to_num_con(d_test, 'clarity', clalis) 

d_test.head(3)

,carat,cut,color,clarity,depth,table
0,0.30,5,3,2,60.0,56.0
1,0.34,5,7,8,62.1,57.0
2,1.57,3,2,4,60.3,58.0


In [5]:
X = norm(d_train.drop(columns=['price']))
y = d_train['price']
X.head(3)

,carat,cut,color,clarity,depth,table
0,0.880093,0.982962,-0.833658,-0.035196,0.877395,-0.205924
1,-1.095267,-0.809998,1.525915,1.177942,1.579543,-0.653290
2,-0.797901,0.086482,0.346128,0.571373,-0.386471,0.241442


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
d_test = norm(d_test)
d_test.head(3)

,carat,cut,color,clarity,depth,table
0,-1.048154,0.978650,-0.805754,-1.238972,-1.203383,-0.651290
1,-0.964994,0.978650,1.521942,2.416515,0.248074,-0.202597
2,1.592168,-0.810376,-1.387678,-0.020476,-0.996032,0.246095


In [8]:
d_train.to_csv('../output/diamonds_train.csv',index=False)
d_test.to_csv('../output/diamonds_test.csv',index=False)

## 4. Select correct parameters

In [74]:
df = h2o.import_file('../output/diamonds_train.csv')
df_test = h2o.import_file('../output/diamonds_test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [75]:
df.head(3)

carat,cut,color,clarity,depth,table,price
1.21,5,3,4,63,57,6134
0.28,3,7,6,64,56,532
0.42,4,5,5,61.2,58,1103


In [76]:
splits = df.split_frame(ratios=[0.8],seed=1)
train = splits[0]
test = splits[1]

In [77]:
len(train)

32378

In [82]:
test.head()

carat,cut,color,clarity,depth,table,price
0.42,4,5,5,61.2,58,1103
1.22,5,2,5,61.9,55.7,6469
0.31,5,6,6,62.4,53,791
2.02,4,6,2,59.8,58,17825
0.33,5,3,3,63,57,610
1.02,4,7,6,61.4,58,10580
0.39,4,5,5,60.6,55,1092
0.33,4,5,4,62.4,58,666
1.28,5,5,5,61.7,55,11580
1.05,2,6,5,63.1,59,7826


In [79]:
y = "price" 
x = df.columns 
x.remove(y) 

In [80]:
aml = H2OAutoML(max_runtime_secs=1200, seed=1)
aml.train(x=x,y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [69]:
lb = aml.leaderboard
lb.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200511_195342,308206,555.163,308206,296.247,0.124841
StackedEnsemble_BestOfFamily_AutoML_20200511_195342,312369,558.9,312369,295.413,0.119341
XGBoost_grid__1_AutoML_20200511_195342_model_1,331905,576.112,331905,315.219,nan
XGBoost_grid__1_AutoML_20200511_195342_model_2,353714,594.739,353714,338.183,nan
XGBoost_3_AutoML_20200511_195342,384995,620.48,384995,353.32,nan
GBM_grid__1_AutoML_20200511_195342_model_1,386010,621.297,386010,346.742,nan
DRF_1_AutoML_20200511_195342,460980,678.955,460980,356.128,0.135493
DeepLearning_1_AutoML_20200511_195342,511258,715.023,511258,413.221,nan
XRT_1_AutoML_20200511_195342,512526,715.909,512526,371.139,0.143863
XGBoost_2_AutoML_20200511_195342,626200,791.328,626200,477.015,nan


In [84]:
pred = aml.leader.predict(test)
pred.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
1060.54
6685.39
810.869
16109
548.68
10348.6
984.505
749.195
10991
7305.62


As this model seem to have pretty good resluts ... let's try it for real!

In [87]:
taml = H2OAutoML(max_runtime_secs=1200, seed=1)
taml.train(x=x,y=y, training_frame=df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [93]:
fpred = taml.leader.predict(df_test)
fpred.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
155.783
659.441
4319.65
155.322
5031.86
158.443
157.037
3824.75
134.716
155.085


In [94]:
dfpred = fpred.as_data_frame()

In [95]:
columns=['id','price']
dfpred = dfpred.reset_index(level=0)
dfpred.columns = columns

In [96]:
dfpred.head(3)

,id,price
0,0,155.782563
1,1,659.440683
2,2,4319.645742


In [97]:
dfpred.to_csv('../output/sample_submission.csv',index=False)